In [ ]:
import urllib,time,datetime
import os,time

class Quote(object):
  DATE_FMT = '%Y-%m-%d'
  TIME_FMT = '%H:%M:%S'
  
  def __init__(self):
    self.symbol = ''
    self.date,self.time,self.open_,self.high,self.low,self.close,self.volume = ([] for _ in range(7))

  def append(self,dt,open_,high,low,close,volume):
    self.date.append(dt.date())
    self.time.append(dt.time())
    self.open_.append(float(open_))
    self.high.append(float(high))
    self.low.append(float(low))
    self.close.append(float(close))
    self.volume.append(int(volume))
      
  def to_csv(self):
    return ''.join(["{0},{1},{2},{3:.2f},{4:.2f},{5:.2f},{6:.2f},{7}\n".format(self.symbol,
              self.date[bar].strftime('%Y-%m-%d'),self.time[bar].strftime('%H:%M:%S'),
              self.open_[bar],self.high[bar],self.low[bar],self.close[bar],self.volume[bar]) 
              for bar in xrange(len(self.close))])
    
  def write_csv(self,filename):
    with open(filename,'w') as f:
      f.write(self.to_csv())
        
  def read_csv(self,filename):
    self.symbol = ''
    self.date,self.time,self.open_,self.high,self.low,self.close,self.volume = ([] for _ in range(7))
    for line in open(filename,'r'):
      symbol,ds,ts,open_,high,low,close,volume = line.rstrip().split(',')
      self.symbol = symbol
      dt = datetime.datetime.strptime(ds+' '+ts,self.DATE_FMT+' '+self.TIME_FMT)
      self.append(dt,open_,high,low,close,volume)
    return True

  def __repr__(self):
    return self.to_csv()

class GoogleIntradayQuote(Quote):
  ''' Intraday quotes from Google. Specify interval seconds and number of days '''
  def __init__(self,symbol,interval_seconds=300,num_days=5):
    super(GoogleIntradayQuote,self).__init__()
    self.symbol = symbol.upper()
    url_string = "http://www.google.com/finance/getprices?q={0}".format(self.symbol)
    url_string += "&i={0}&p={1}d&f=d,o,h,l,c,v".format(interval_seconds,num_days)
        
    csv = urllib.urlopen(url_string).readlines()
    for bar in xrange(7,len(csv)):
      if csv[bar].count(',')!=5:
        continue
        
      offset,close,high,low,open_,volume = csv[bar].split(',')
     
      if offset[0]=='a':
        day = float(offset[1:])
        offset = 0
      else:
        offset = float(offset)
    
      open_,high,low,close = [float(x) for x in [open_,high,low,close]]
      dt = datetime.datetime.fromtimestamp(day+(interval_seconds*offset))
      self.append(dt,open_,high,low,close,volume)

In [ ]:
# Set system time to EST time zone
os.environ['TZ'] = 'EST+05EDT,M4.1.0,M10.5.0'
time.tzset()

In [ ]:
# Set scrapping time frame and data granularity level:
interval_seconds = 60
num_days = 10

In [ ]:
# Start Scrapping every day after 4:00 PM on two different AWS EC2

# To Avoid IP Blocking:
pause_count = 500 # number of loops
pause_counter = pause_count
pause_duration = 3600 # sleep 60 minutes after 500 scraps

# for exchange in ['nasdaq']:
# for exchange in ['amex']:
for exchange in ['nyse']:  
    
    ticker_source = open('Ticker_Only/correct_'+exchange+'.txt', 'r')

    for ticker in ticker_source:
        ticker = ticker.strip()
        temp_file = open('Stock_Intraday_Data/'+exchange+'/'+ticker+'.csv','w')
        retrieved_result = GoogleIntradayQuote(ticker, interval_seconds, num_days)
        print ticker
        temp_file.write(retrieved_result.to_csv())
        temp_file.close()
        
        if pause_counter > 0:
            pause_counter -= 1
        else:
            print 'Pause seconds: ' + str(pause_duration) + '...'
            time.sleep(pause_duration)
            print 'Resume Scrapping!'
            pause_counter = pause_count
            
    print exchange + ' has done!'